In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  2 12:56:02 2022

@author: steph

Our simulation consists of 2 stocks 
we simulate their drift, volatility, and transition matrix
and plug them in max likelihood
before testing using MCMCs
"""

import numpy as np
import pandas as pd

##############################################################################
### VARIABLES ###
# drift = normal distribution(mean, sigma)
# dim(drift) = [stock] [state] 
# list of mean of stocks : [stock1, stock2]
mean_stocks = [0,0]
# list of sigma of stocks : [stock1, stock2]
sigma_stocks = []
sigma_states_elements = [0.5, 0.2, 0.5]

# randomly generated 2 stocks
num_stocks = 2

# we have 3 states: 
# state 0: crisis, state 1: stable, state 2: bubble
num_state = 3

# how many time (index of time)
N = 50

#fix the random seed
# np.random.seed(0)
#############################################################################
# b : drift for each stocks, for each state
# ex: b[0,1] is drift for stock 0 and state 1
b = np.random.randn(num_stocks, num_state)

# b0 < b1 < b2
for j in range(num_stocks):
    for i in range(num_state):
        if i == 0:
            b[j,i] = np.random.randn(1)
        else: 
            # make sure b of the next state is larger than the one before
            # b0 (crisis period) < b1 (stable) < b2 (bubble periode)
            b[j,i] = np.random.normal(b[j,i-1] +1, 0.5,1) 


print("drift matrix",b)

drift matrix [[2.38344536 2.70944148 3.92138499]
 [0.99470193 2.15879448 3.57123947]]


In [4]:
# volatility
sigma_stocks = [
    np.full((num_stocks, num_stocks), sigma_states_elements[x], dtype=np.float32)
    for x in range(num_state)
]
for x in sigma_stocks:
    np.fill_diagonal(x, 1)

sigma_stocks

[array([[1. , 0.5],
        [0.5, 1. ]], dtype=float32),
 array([[1. , 0.2],
        [0.2, 1. ]], dtype=float32),
 array([[1. , 0.5],
        [0.5, 1. ]], dtype=float32)]

In [6]:
# create 3 matrix of dxd matrix
# for state 1 & 3 -> similar covariance matrix

# transition matrixs 
# P [stock][from state][to state]
P = [[0.3, 0.4, 0.3], [0.1, 0.9, 0.0], [0.4, 0.3, 0.3]], \
    [[0.2, 0.6, 0.2], [0.01, 0.9, 0.09], [0.4, 0.3, 0.3]]

P

([[0.3, 0.4, 0.3], [0.1, 0.9, 0.0], [0.4, 0.3, 0.3]],
 [[0.2, 0.6, 0.2], [0.01, 0.9, 0.09], [0.4, 0.3, 0.3]])

In [9]:
# simulate y : state matrix
# t = 0 : first state : random between state 1,2,3
# y[i,t] : stock i, and t time
# ex: y[0,1] : state at stock 0 when time is in index 1.  

y = np.zeros((num_stocks, N))

for i in range(num_stocks):
    for t in range(N):
        if t == 0:
            y[i,t] = np.random.randint(0,3)
        else: 
            prev_state = int(y[i, t-1])
            y[i,t] = np.random.choice( np.arange(0,3), p=[P[i][prev_state][0], \
                                                          P[i][prev_state][1], \
                                                          P[i][prev_state][2]] )
y = y.astype('int') # state is always integer
print("state matrix", y)

state matrix [[2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 2 2 1 1 1 1 1 1 1 0 0
  1 0 1 1 1 1 1 1 1 1 1 1 1 1]
 [2 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 1
  2 0 1 1 1 1 1 1 1 1 1 0 0 1]]


In [12]:
# DELTA of BROWNIAN MOTION is a normal distribution of mean 0 and std 1
# 1 stock have 1 brownian motion
# the length of 1 brownian motion is N
# brownian motion have dim(num_stock, N)

brownian_motion_delta = np.zeros((num_stocks, N))
for j in range(num_stocks): 
    brownian_motion_delta[j] = np.random.standard_normal(N)

brownian_motion_delta

array([[-0.36657129, -0.23457052, -2.2359985 ,  0.14330138,  1.20250644,
        -0.57050534,  1.1783647 , -0.20309353, -0.12424334,  0.5076694 ,
        -0.03441084, -1.10492211, -0.62209761, -0.48262235,  0.45061878,
         1.27319985, -0.1367933 ,  0.64850603, -0.38801925, -0.54928661,
         0.12715166, -0.99799794, -1.12698189, -1.11110072, -0.55062818,
        -0.13733877, -0.07495608,  0.97736925, -0.69615047,  0.89735194,
        -0.12555587,  0.45373972, -1.9584951 ,  0.41813168, -0.22524453,
        -0.04630085, -1.35509915,  0.2514871 , -1.17043295, -0.07641419,
        -1.09901774,  1.37743289,  0.03983347,  0.06043545,  0.11947394,
        -2.64931412, -1.21448495,  0.1881905 , -0.07219409,  0.61956638],
       [-0.46373344,  0.1192925 ,  0.18092133,  0.11710875, -0.11589008,
        -1.49688262, -0.02916886,  0.71627083,  0.32152218, -0.37671539,
         0.07573479,  2.24644151,  0.18307535, -0.28557097,  1.58348095,
        -0.25047401, -1.6110309 ,  0.09369523, -0.

In [13]:
# r : rate of Return of stock price from timme 0 to time t
r = np.zeros((num_stocks, N))
for i in range(num_stocks):
    for n in range(1,N+1):
        sum_b = 0
        sum_sigma = 0
        for t in range(n):
            state_now = y[i,t]
            sum_b = sum_b + b[i , state_now]
            for d in range(num_stocks):
                sum_sigma = sum_sigma + (sigma_stocks[state_now][i][d] * brownian_motion_delta[i, t])
            r[i,t] = sum_b + sum_sigma
print("return matrix", r)


return matrix [[  3.37152805   5.79948491   5.82572818   8.70713132  12.85958053
   14.8844156   19.00789472  21.47362396  24.03397343  27.35261819
   30.02076666  31.40430161  33.36722595  35.49752062  38.74770463
   42.98498594  45.53027546  49.01792418  51.26174256  53.3120401
   56.17406358  57.68590753  59.04297074  59.75976502  62.85520774
   66.57058458  70.37953546  74.26182004  76.13588096  79.92214477
   82.4809192   85.73484834  86.09409569  89.30529519  91.35087376
   93.66486783  94.74819033  97.50886633  98.81378826 101.43153271
  102.8221529  107.18451385 109.9417555  112.72371952 115.57652973
  115.10679426 116.35885379 119.29412388 121.91693245 125.36985358]
 [  2.87563931   4.04928      6.42518008   8.72450506  12.12190942
   12.48444475  14.6082366   17.62655607  20.17117717  21.87791319
   24.12758942  28.98211373  31.36059863  33.17670794  37.23567957
   39.09390524  39.31946263  41.59069139  43.21183977  45.56571777
   46.68878571  48.69266709  50.82503023  55.585

In [14]:
# Price matrix S
# dimension: number_of_stock x N times
# ex: S[0,1] : stock 0 , time 1
S = np.zeros((num_stocks, N))
for i in range(num_stocks):
    for n in range(N):
        if n == 0:
            S[i,n] = np.random.randint(50,100)
        else:
            S[i,n] = S[i, n-1]*r[i,n]
print("price matrix", S)

price matrix [[8.70000000e+01 5.04555187e+02 2.93940137e+03 2.55937538e+04
  3.29124938e+05 4.89883235e+06 9.31164896e+07 1.99954848e+09
  4.80570951e+10 1.31448737e+12 3.94619187e+13 1.23927400e+15
  4.13511355e+16 1.46786279e+18 5.68763136e+19 2.44482754e+21
  1.11313671e+23 5.45636511e+24 2.79702783e+26 1.49115260e+28
  8.37641010e+29 4.83200818e+31 2.85296118e+33 1.70492290e+35
  1.07163283e+37 7.13392238e+38 5.02082143e+40 3.72855338e+42
  2.83876696e+44 2.26880344e+46 1.87132993e+48 1.60438188e+50
  1.38127807e+52 1.23355446e+54 1.12686278e+56 1.05547453e+58
  1.00004302e+60 9.75130608e+61 9.63563494e+63 9.77357221e+65
  1.00493974e+68 1.07713977e+70 1.18422637e+72 1.33490402e+74
  1.54283574e+76 1.77590876e+78 2.06642707e+80 2.46512607e+82
  3.00540609e+84 3.76787321e+86]
 [5.80000000e+01 2.34858240e+02 1.50900648e+03 1.31653347e+04
  1.59588995e+05 1.99237999e+06 2.91051582e+07 5.13023704e+08
  1.03482920e+10 2.26399035e+11 5.46246295e+12 1.58313722e+14
  4.96481311e+15 1.64716